In [11]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
    !pip install -q transformers
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/MyDrive/gan_bert_for_cpv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from transformers import logging
logging.set_verbosity_error()
import torch
from create_dataset import CreateDataset
from CPVDataset import CPVDataset
from process_data import ProcessData
from utils import *
from torch.utils.data import DataLoader
from run_model import RunModel
from models import BertGenerator, BertDiscriminator,EmbeddingClassifier
from transformers import AutoTokenizer, AutoModel, AutoConfig

In [13]:
label_column = "categorie"
text_column = "Korte beschrijving aanbesteding"
input_df = pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_dutch_ted1.csv")
input_df2=pd.read_csv("/content/drive/MyDrive/gan_bert_for_cpv/data/cleaned_foreign_ted1.csv")
all_input=pd.concat([input_df,input_df2])

In [14]:
all_input=all_input.sample(frac=1).groupby(label_column, sort=False).head(2000)

In [15]:
running_device = torch.device("cuda")

In [16]:
prd = ProcessData(all_input, label_column, text_column, sample_num=200, aug_limit=200,test_per_label_num=30, drop_limit=100,
                  balanced_test=True)
test_df, suf_df, insuf_df, label_encoder, onehot = prd.process()
# balanced_test=test_df.sample(frac=1).groupby(label_column, sort=False).head(30)
value_counts = insuf_df[label_column].value_counts()
model_runner = RunModel(running_device)

After removing duplicate sentence: 259256
After drop cpv with low observations we have : 227911
The num of class is  624
The number of total data is:  112969
Total number of test data is:  18720
Total number of train data is:  91830


In [17]:
value_counts

Speciale optische instrumenten                               170
Onderhoud en reparatie van computeruitrusting                170
Algemene infectiewerende middelen voor systemisch gebruik    170
Computerondersteuningsdiensten                               170
Ziekenhuisdiensten                                           170
                                                            ... 
Diensten voor technische analyse of adviezen                  70
Huishoudelijke apparaten                                      70
Industriële gassen                                            69
Uitrusting voor gymnastiekzaal                                67
Drinkwaterdistributie                                         66
Name: categorie, Length: 624, dtype: int64

In [18]:
from joblib import dump, load
dump(onehot, '/content/drive/MyDrive/gan_bert_for_cpv/output/category/onehot.joblib')

['/content/drive/MyDrive/gan_bert_for_cpv/output/category/onehot.joblib']

In [19]:
dump(label_encoder, '/content/drive/MyDrive/gan_bert_for_cpv/output/department/labelencoder.joblib')

['/content/drive/MyDrive/gan_bert_for_cpv/output/department/labelencoder.joblib']

In [20]:
tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
config = AutoConfig.from_pretrained("GroNLP/bert-base-dutch-cased", output_hidden_states=True)
bert = AutoModel.from_pretrained("GroNLP/bert-base-dutch-cased", config=config).to(running_device)

In [ ]:
cd = CreateDataset(text_column, label_column, label_encoder, onehot, 9, running_device, max_length=100)
train_dataset=cd.create(insuf_df)
test_dataset=cd.create(test_df)

Init new bert


Getting bert hidden states from layer: 9: 100%|██████████| 91830/91830 [19:46<00:00, 77.43it/s]


In [ ]:
test_dl = DataLoader(test_dataset, shuffle=True, batch_size=20)
train_dl = DataLoader(train_dataset, shuffle=True, batch_size=20)

In [ ]:
from models import HiddenClassifier9
classifier = HiddenClassifier9(prd.num_class, bert)
classifier = model_runner.train_classifier(train_dl, test_dl, classifier, lr=2e-5, n_epochs=3)
predictions, labels = model_runner.predict(test_dl, classifier)
acc, f1, precision, recall, label_f1 = evaluate(labels, predictions)

In [ ]:
classifier = model_runner.train_classifier(gan_aug_dl, test_dl, classifier, lr=3e-5, n_epochs=5)
torch.save(classifier,'/content/drive/MyDrive/gan_bert_for_cpv/output/department/model2')